In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_recall_fscore_support
from sklearn.preprocessing import label_binarize

In [2]:
# 1. 데이터 불러오기
data = pd.read_csv('azdigar nav sartirovka+0 delete.csv')

In [3]:
# 분석할 열들
columns = [
    'w08chronic_a', 'w08chronic_b', 'w08chronic_c',
    'w08chronic_d', 'w08chronic_e', 'w08chronic_f',
    'w08chronic_g', 'w08chronic_h', 'w08chronic_i',
    'w08chronic_k', 'w08chronic_l', 'w08chronic_m'
] 


In [4]:
# 2. 데이터 클리닝 - 'w08chronic_m'에서 값이 3인 행 제거
cleaned_data = data[data['w08chronic_m'] != 3] 


In [5]:
# 3. 피처 데이터 정의 (모든 chronic 컬럼 제외)
X = cleaned_data.drop(columns=columns)
y_all = cleaned_data[columns]


In [6]:
# 4. 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [7]:
# 5. 모델 정의
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
}

In [8]:
# 6. 유전 알고리즘 파라미터 설정
population_size = 10  # 개체군 크기 
num_generations = 10  # 세대 수 
mutation_rate = 0.1  # 돌연변이 확률

# 초기 개체군 생성
def initialize_population():
    return [np.random.randint(2, size=X_scaled.shape[1]).tolist() for _ in range(population_size)]

# 적합도 평가 함수
def fitness_function(individual, model, X_train, y_train, X_test, y_test):
    selected_features = [i for i, bit in enumerate(individual) if bit]
    if len(selected_features) == 0:
        return 0  # 피처가 선택되지 않으면 적합도는 0
    
    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]
    
    try:
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        # 타겟 클래스가 두 개 이상이면 AUC 계산을 위해 타겟을 이진화 (One-vs-Rest 방식)
        y_test_bin = label_binarize(y_test, classes=np.unique(y_train))
        if y_test_bin.shape[1] == 1:  # 클래스가 하나인 경우
            y_test_bin = np.concatenate([1 - y_test_bin, y_test_bin], axis=1)
        
        f1 = f1_score(y_test, y_pred, average='macro')
        auc = roc_auc_score(y_test_bin, model.predict_proba(X_test_selected), multi_class="ovr", average="macro")
        
        return f1  # 적합도는 F1 점수로 설정
    except Exception as e:
        print(f"적합도 평가 중 오류 발생: {e}")
        return 0

# 개체군 선택 함수
def selection(population, model, X_train, y_train, X_test, y_test):
    scores = [
        _function(ind, model, X_train, y_train, X_test, y_test) for ind in population]
    # 상위 절반 선택
    selected_indices = np.argsort(scores)[-population_size // 2:]
    return [population[i] for i in selected_indices]

# 교차 함수
def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

# 돌연변이 함수
def mutate(individual):
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            individual[i] = 1 - individual[i]  # 비트 반전
    return individual

# 유전 알고리즘 실행 함수
def run_genetic_algorithm(model, X_train, y_train, X_test, y_test):
    population = initialize_population()
    best_individual = None
    best_f1 = -1

    for generation in range(num_generations):
        print(f"Generation {generation + 1}/{num_generations}")  # 진행 상황 출력
        selected_population = selection(population, model, X_train, y_train, X_test, y_test)
        
        children = []
        while len(children) < population_size - len(selected_population):
            parent1, parent2 = random.sample(selected_population, 2)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1)
            child2 = mutate(child2)
            children.extend([child1, child2])
        
        # 새로운 개체군 구성
        population = selected_population + children[:population_size - len(selected_population)]
        
        # 현재 세대에서 최고의 개체 찾기
        for ind in population:
            current_f1 = fitness_function(ind, model, X_train, y_train, X_test, y_test)
            if current_f1 > best_f1:
                best_f1 = current_f1
                best_individual = ind

    return best_individual


In [9]:
# 7. 결과 저장을 위한 빈 데이터프레임 생성
result_table = pd.DataFrame(columns=[
    'Target', 'Model', 'Precision', 'Recall', 'F1-Score', 
    'Accuracy', 'Micro F1', 'Macro F1', 'Weighted F1', 'AUC'
]) 

In [10]:
# 8. 각 타겟 열을 사용한 학습 및 평가
for target_column in columns:
    print(f'\n[{target_column}] 열을 예측:')
    print('==================================')

    y = cleaned_data[target_column].values

    # 데이터 나누기
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42
    )

    # 9. 각 모델에 대해 학습 및 평가
    for model_name, model in models.items():
        print(f'\n[{model_name}] 모델 평가:')
        print('--------------------------')

        try:
            # 유전 알고리즘을 사용한 피처 선택
            best_individual = run_genetic_algorithm(model, X_train, y_train, X_test, y_test)
            selected_features = [i for i, bit in enumerate(best_individual) if bit]

            if len(selected_features) == 0:
                print(f'선택된 피처가 없습니다. {target_column} 열에 대한 {model_name} 모델 평가를 건너뜁니다.')
                continue

            X_train_selected = X_train[:, selected_features]
            X_test_selected = X_test[:, selected_features]

            # 타겟 클래스가 두 개 이상이면 AUC 계산을 위해 타겟을 이진화 (One-vs-Rest 방식)
            y_test_bin = label_binarize(y_test, classes=np.unique(y))
            if y_test_bin.shape[1] == 1:  # 클래스가 하나인 경우
                y_test_bin = np.concatenate([1 - y_test_bin, y_test_bin], axis=1)

            # 모델 학습
            model.fit(X_train_selected, y_train)

            # 예측
            y_pred = model.predict(X_test_selected)

            # 확률 예측 값이 있는 경우
            if hasattr(model, "predict_proba"):
                y_pred_proba = model.predict_proba(X_test_selected)
            else:
                y_pred_proba = np.zeros((len(y_test), len(np.unique(y_test))))  # 확률 값을 지원하지 않는 모델에 대한 처리

            # 정확도 계산
            accuracy = accuracy_score(y_test, y_pred)

            # precision, recall, f1-score 계산 (weighted average)
            precision, recall, f1, _ = precision_recall_fscore_support(
                y_test, y_pred, average='weighted'
            )

            # Micro, Macro, Weighted F1 계산
            micro_f1 = f1_score(y_test, y_pred, average='micro')
            macro_f1 = f1_score(y_test, y_pred, average='macro')
            weighted_f1 = f1_score(y_test, y_pred, average='weighted')

            # AUC 계산
            try:
                auc = roc_auc_score(
                    y_test_bin, y_pred_proba, multi_class="ovr", average='weighted'
                )
            except ValueError:
                auc = np.nan

            # 결과 추가
            new_row = pd.DataFrame({
                'Model': [model_name],
                'Target': [target_column],
                'Precision': [precision],
                'Recall': [recall],
                'F1-Score': [f1],
                'Accuracy': [accuracy],
                'Micro F1': [micro_f1],
                'Macro F1': [macro_f1],
                'Weighted F1': [weighted_f1],
                'AUC': [auc]
            })
            result_table = pd.concat([result_table, new_row], ignore_index=True)

        except Exception as e:
            print(f"{model_name} 모델에서 오류 발생: {e}")


[w08chronic_a] 열을 예측:

[Random Forest] 모델 평가:
--------------------------
Generation 1/100
Generation 2/100
Generation 3/100


In [1]:
# 11. 결과 테이블 출력
print("\n전체 모델 평가 결과:")
print(result_table)
result_table.to_csv("okok.csv")


전체 모델 평가 결과:


NameError: name 'result_table' is not defined